In [1]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

from fenics import *
from mshr import *

--2023-12-14 13:10:34--  https://fem-on-colab.github.io/releases/fenics-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.06K  --.-KB/s    in 0s      

2023-12-14 13:10:34 (51.7 MB/s) - ‘/tmp/fenics-install.sh’ saved [4156/4156]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/2a03579/releases/pybind11-install.s

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load mesh
# mesh = UnitCubeMesh(16, 16, 16)
domain0 = Box(Point(0, 0, 0), Point(100, 20, 50))
domain1 = Box(Point(0, 20, 20), Point(100, 60, 30))
domain = domain0 + domain1
mesh = generate_mesh(domain, 32)

# Define function spaces
basis_order = 2
V = FunctionSpace("CG", mesh.ufl_cell(), basis_order)

V_0 = FunctionSpace(mesh, 'P', 1)

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_cdd21d347a8b939a40c5cd04a75bdb0e0e4b85cb

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00574136 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.314795 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representation
INFO:FFC:----------------------------

In [50]:
t_end = 10
dt = 0.1
T_in = 400
T_out = 350
T_inf = 300

k = Constant(45) # steel conductivity coefficient w/m.k
h = Constant(40) # air convection coefficient w/m^2.k
q = Constant(-250)                    # heat flux w/m^2


In [51]:
# Boundaries
# Define boundary condition
tol = 1E-14

# defining boundary markers
boundary_markers = MeshFunction("size_t", mesh, mesh.topology().dim()-1, 0)

class Left(SubDomain):
  tol = 1E-14
  def inside(self, x, on_boundary):
    return on_boundary and near(x[0], 0, tol)

class Right(SubDomain):
  tol = 1E-14
  def inside(self, x, on_boundary):
    return on_boundary and near(x[0], 100, tol)

class Bottom(SubDomain):
  tol = 1E-14
  def inside(self, x, on_boundary):
    return on_boundary and near(x[1], 0, tol)

class Top(SubDomain):
  tol = 1E-14
  def inside(self, x, on_boundary):
    return on_boundary and near(x[1], 60, tol)

class Side(SubDomain):
    tol = 1E-14
    def inside(self, x, on_boundary):
        return on_boundary and not (near(x[0], 0, tol) or near(x[0], 100, tol) or near(x[1], 0, tol) or near(x[1], 60, tol))

bx0 = Left()
bx1 = Right()
by0 = Bottom()
by1 = Top()
bside = Side()

bx0.mark(boundary_markers, 1)    # Dirichlet x = 0
bx1.mark(boundary_markers, 2)    # Dirichlet x = 100
by0.mark(boundary_markers, 3)    # Neumann   y = 0
by1.mark(boundary_markers, 4)    # Dirichlet y = 70
bside.mark(boundary_markers, 5)  # Robin     Side

# defining boundary conditions
bc0 = DirichletBC(V_0, T_in, boundary_markers, 1)
bc1 = DirichletBC(V_0, T_out, boundary_markers, 2)
bc2 = DirichletBC(V_0, (T_out+T_in)/2, boundary_markers, 4)
bcs = [bc0, bc1, bc2]



In [52]:
################# STEADY HEAT TRANSFER PROBLEM ###################

# defining ds and dx of boundary markers
ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)
dx = Measure('dx', domain=mesh)

# Define variational problem
u = TrialFunction(V_0)
v = TestFunction(V_0)

a = k*dot(grad(u), grad(v))*dx + h*u*v*ds(5)
L = - q*v*ds(3) + h*T_inf*v*ds(5)

#################################

# Compute solution
u = Function(V_0)
solve(a == L, u, bcs)

In [53]:
################ TRANSIENT HEAT TRANSFER PROBLEM ##############

# Define variational problem
T = TrialFunction(V_0)
v = TestFunction(V_0)
T_n = Function(V_0)

# Define initial condition
T_init = Expression('300.0',degree=2)
T_n = interpolate(T_init, V_0)


F = T*v*dx + dt*k*dot(grad(T), grad(v))*dx - T_n*v*dx + dt*h*T*v*ds(5) + dt*q*v*ds(3) - dt*h*T_inf*v*ds(5)
a = lhs(F)
L = rhs(F)

T = Function(V_0)
t = 0

num_steps = int(t_end/dt)
# Save solution in VTK format
ufile_pvd0 = File('/content/drive/MyDrive/project_one/3D_Heattransfer/Results/Temp_3d_transient.pvd')


for n in range (num_steps):
  t += dt
  solve(a==L , T, bcs)
  T_n.assign(T)
  ufile_pvd0 << (T,t)

In [54]:
# Get sub-functions
output_dir = '/content/drive/MyDrive/project_one/3D_Heattransfer/Results/'

# Save solution in VTK format
ufile_pvd = File(output_dir + "Temp_3d_corrected.pvd")
ufile_pvd << u
